In [1]:
# s - single solution
# p - population

# NEAT

# intenzifikacija - intenzivnije bolje
# diverzifikacija - diversity 

# jedinka: niz bitova - 0101000111

# roditelj1 - 0101000111
# roditelj2 - 1011110100

# dete1 - 1101001100
# dete2 - 1011110111
    
# inicijalizuj populaciju - niz jedinki
# while not stop_condition:
#     if elitizam:
#         iskopiraj u novu populaciju najboljih nekoliko jedinki
    
#     izaberi dobre jedinke za reprodukciju
#     ukrstanje
#     mutacija
#     dobili novu populaciju (population = newPopulation)
    
# najbolja jedinka iz poslednje populacije

In [15]:
# UFLP
# I korisnika
# J resursa
# c_ij - 
# f_j -

# resenje - koje bolnice treba da se sagrade
# kodiranje: 1000001000
# fitnes: suma c_ij i f_j
# hocemo sto manji fitnes

In [16]:
import random

In [24]:
class Individual():
    
    def __init__(self, numUsers, numResources, cost, fixedCost):
        self.code = [random.random() < 0.25 for _ in range(numResources)]
        self.correctNonFeasible()
        
        self.fitness = self.calculateFitness(cost, fixedCost)
        
    def __lt__(self, other):
        return self.fitness < other.fitness
        
    def invert(self):
        i = random.randrange(len(self.code))
        self.code[i] = not self.code[i]
        if self.isFeasible():
            return i
        return -1
    
    def isFeasible(self):
        for c in self.code:
            if c:
                return True
        return False
        
    def correctNonFeasible(self):
        for c in self.code:
            if c:
                return
#         jeste nedopustivo
        index = random.randrange(0, len(self.code))
        self.code[index] = True
        
    def calculateFitness(self, cost, fixedCost):
        numUsers = len(cost)
        numResources = len(fixedCost)
        value = 0.0
        used = [False for _ in range(numResources)]
        
        for i in range(numUsers):
            minCost = float('inf')
            usedResource = -1
            for j in range(numResources):
                if self.code[j] and cost[i][j] < minCost:
                    minCost = cost[i][j]
                    usedResource = j
            value += minCost
            used[usedResource] = True
        
        for i in range(numResources):
            if used[i]:
                value += fixedCost[i]
                
        self.code = used
        return value

In [25]:
def readInput(filename):
    with open(filename, 'r') as f:
        numUsers, numResources = [int(x) for x in f.readline().split()]
        cost = [[int(x) for x in f.readline().split()] for i in range(numUsers)]
        fixedCost = [int(x) for x in f.readline().split()]
        
        return numUsers, numResources, cost, fixedCost

In [26]:
numUsers, numResources, cost, fixedCost = readInput('../03/uflp1.txt')

In [27]:
def selection(population):
    TOURNAMENT_SIZE = 6
    minCost = float('inf')
    bestIndex = -1
    
    for i in range(TOURNAMENT_SIZE):
        index = random.randrange(len(population))
        if population[index].fitness < minCost:
            minCost = population[index].fitness
            bestIndex = index
    
    return bestIndex

In [28]:
def crossover(parent1, parent2, child1, child2):
    breakpoint = random.randrange(0, len(parent1.code))
    
    child1.code[:breakpoint] = parent1.code[:breakpoint]
    child2.code[:breakpoint] = parent2.code[:breakpoint]
    
    child1.code[breakpoint:] = parent2.code[breakpoint:]
    child2.code[breakpoint:] = parent1.code[breakpoint:]
    
#     00001110
#     11010000
    child1.correctNonFeasible()
    child2.correctNonFeasible()

In [29]:
def mutation(individual):
    MUTATION_PROB = 0.05
    for i in range(len(individual.code)):
        if random.random() < MUTATION_PROB:
            individual.code[i] = not individual.code[i]
# 0001000
    individual.correctNonFeasible()

In [34]:
POPULATION_SIZE = 100
population = [Individual(numUsers, numResources, cost, fixedCost) for i in range(POPULATION_SIZE)]
newPopulation = [Individual(numUsers, numResources, cost, fixedCost) for i in range(POPULATION_SIZE)]

ELITISM_SIZE = int(0.3 * POPULATION_SIZE)
MAX_ITER = 500
for i in range(MAX_ITER):
    population.sort()
    newPopulation[:ELITISM_SIZE] = population[:ELITISM_SIZE]
    for j in range(ELITISM_SIZE, POPULATION_SIZE, 2):
        parent1Index = selection(population)
        parent2Index = selection(population)
        
        crossover(population[parent1Index], population[parent2Index], newPopulation[j], newPopulation[j+1])
        
        mutation(newPopulation[j])
        mutation(newPopulation[j+1])
        
        newPopulation[j].fitness = newPopulation[j].calculateFitness(cost, fixedCost)
        newPopulation[j + 1].fitness = newPopulation[j + 1].calculateFitness(cost, fixedCost)
        
    population = newPopulation
        
bestIndividual = min(population, key=lambda x: x.fitness)
print('Solution: {}, fitness: {}'.format(bestIndividual.code, bestIndividual.fitness))

Solution: [True, False, False], fitness: 34.0


In [35]:
def simulatedAnnealing(individual, iters, cost, fixedCost):
    for i in range(iters):
#         izmeni malo trenutno resenje - individual.code
        j = individual.invert()
        if j < 0:
            continue
        newFitness = individual.calculateFitness(cost, fixedCost)
        
        if newFitness < individual.fitness:
            individual.fitness = newFitness
        else:
            p = 1.0 / (i + 1) ** 0.5
            q = random.uniform(0, 1)
            if p > q:
#                 prihvati losije resenje
                individual.fitness = newFitness
            else:
#             vrati se na prethodno resenje
                individual.code[j] = not individual.code[j]

In [36]:
POPULATION_SIZE = 100
population = [Individual(numUsers, numResources, cost, fixedCost) for i in range(POPULATION_SIZE)]
newPopulation = [Individual(numUsers, numResources, cost, fixedCost) for i in range(POPULATION_SIZE)]

ELITISM_SIZE = int(0.3 * POPULATION_SIZE)
MAX_ITER = 500
for i in range(MAX_ITER):
    population.sort()
    newPopulation[:ELITISM_SIZE] = population[:ELITISM_SIZE]
    for j in range(ELITISM_SIZE, POPULATION_SIZE, 2):
        parent1Index = selection(population)
        parent2Index = selection(population)
        
        crossover(population[parent1Index], population[parent2Index], newPopulation[j], newPopulation[j+1])
        
        mutation(newPopulation[j])
        mutation(newPopulation[j+1])
        
        newPopulation[j].fitness = newPopulation[j].calculateFitness(cost, fixedCost)
        newPopulation[j + 1].fitness = newPopulation[j + 1].calculateFitness(cost, fixedCost)
        
    
    simulatedAnnealing(newPopulation[0], 10, cost, fixedCost)
    population = newPopulation
        
bestIndividual = min(population, key=lambda x: x.fitness)
print('Solution: {}, fitness: {}'.format(bestIndividual.code, bestIndividual.fitness))

Solution: [True, False, False], fitness: 34.0
